# FragMap_CNN Tutorial

###TODO:
Need to prepare a simple tutorial markdown(.md), where we simply demonstrate what is being done here, including all functionalities, visualizations etc. Unit tests have perfect examples. The best way to do this would be using .ipynb and save it as .md file

## Objective:
Demonstrate functionalities and visualizations using `unit/` directory, which contains python unit tests.

### Main Requirements:
- `pytorch` neural net learning framework
- `torchproteinlibrary` lib for handling 3D protein data
- `pyvista` for simple unit tests

In [1]:
import os
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pyvista as pv

from torchsummary import summary

In [2]:
!nvidia-smi

Fri Dec 13 21:53:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:58:00.0 Off |                    0 |
| N/A   72C    P0   172W / 250W |   2982MiB / 12198MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   70C    P0   165W / 250W |   1409MiB / 12198MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
!which nvcc
!which gcc
!conda list | grep torch

/opt/sw/packages/cuda/10.0/bin/nvcc
/projects/community/gcc/7.3.0/gc563/bin/gcc
pytorch                   1.2.0           py3.7_cuda10.0.130_cudnn7.6.2_0    pytorch
torchproteinlibrary       0.1                      pypi_0    pypi
torchsummary              1.5.1                    pypi_0    pypi
torchvision               0.4.0                py37_cu100    pytorch


In [4]:
!conda env list

# conda environments:
#
base                  *  /home/sb1638/anaconda3



In [5]:
!xclock

Error: Can't open display: 


In [6]:
!squeue -u sb1638

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          77364191      main   m_jupy   sb1638  R    2:47:53      1 pascal010


In [7]:
cur_dev = torch.cuda.current_device()

In [8]:
torch.cuda.get_device_name(cur_dev)

'Tesla P100-PCIE-12GB'

### Files

In [9]:
!ls unit

augment.py  cnn.py  mapIO.py  rot.py  target.py  util.py  volume.py


### Step 0: Build the CNN

`src/cnn.py` is imported as:
```python
from src.cnn import CnnModel, count_parameter
```
This code builds the CNN model using pytorch. 

Structure:
- 4 convolutional layers
- Input -> output halved each convolution
- E.g. 48 in -> 24 out, 24 in -> 12 out... etc

Initialized parameters:

- Correct the dimensions of 3D input (pad rectangular boxes to cubic boxes)
- 3D CNN input/output channels, kernal size, and padding; per each layer.


### cnn.py example:

In [10]:
from src.cnn import CnnModel, count_parameters

#### Initialize 11 input channels and depth, width, height

In [11]:
c, d, h, w = 11, 20, 20, 20
torch.manual_seed(3000)

#### Generate some random data

In [12]:
data = torch.randn(1, c, d, h, w)   

print(data.size())

torch.Size([1, 11, 20, 20, 20])


#### Build model using random data

In [13]:
#invoke the model
model = CnnModel()
output = model(data)

#compare input and output shapes
print("11 input channels VS 4 output channels")
print("Input dimension -->", data.size())
print("Output dimension -->",output.size())

11 input channels VS 4 output channels
Input dimension --> torch.Size([1, 11, 20, 20, 20])
Output dimension --> torch.Size([1, 6, 20, 20, 20])


#### NOTE: this is how pytorch will check the parameter count
- The values pertain to multiplicative sums of 3D kernal and layer dimensions

In [14]:
n_params = [p.numel() for p in model.parameters() if p.requires_grad]
print('total =', n_params)

total = [14256, 48, 48, 48, 31104, 24, 24, 24, 7776, 12, 12, 12, 1944, 6, 6, 6]


#### Notice: Pytorch default behavior
- 48 or 48x3 output channels from Conv layer 1
- Then 24x3 output chanels for Conv layer 2
- The pattern continues for the following Conv layers, but only Conv layer 1 does not get multiplied by dimension.

In [ ]:
# estimate number of parameters
est_params = count_parameters(model)
n_layer = [11, 48, 24, 12, 6]
n_params = 0
k_size = 3
for i in range(len(n_layer)-1):
#     if i < 1:
#         n_params += (k_size**3*n_layer[i]+ 1 )*n_layer[i+1]
#     else:
    n_params += (k_size**3*n_layer[i]+ 3 )*n_layer[i+1]



print("Number of parameters in model", n_params)
print("Number of parameteres estimated",est_params )   
if(est_params != n_params):
    print("Check ConvNet structure: kernel size, and # of Conv units in each layer")


# #plot output density map
# chan_id = 0 # can be 0,1,2,3
# channel = output[0,chan_id,:,:,:].detach().numpy()
# p = pv.BackgroundPlotter(point_smoothing = True, off_screen=True)
# p.add_volume(np.abs(channel), cmap = "viridis", opacity = "linear")
# p.show_bounds()

# plotter.show(screenshot='test.png')

Number of parameters in model 55350
Number of parameteres estimated 55350
